In [75]:
#input data 전처리
from konlpy.tag import Kkma
kkma = Kkma()
input_text = '스마트폰'
input_sim = 50  # input data 유사성 민감도 지정 / 숫자가 작을수록 관련 없는게 나올 확률이 커짐 / 최소 50이상 설정
input_text_list = input_text.split(' ') # input data 띄어쓰기로 나누기
noun_list = kkma.nouns(input_text)

# kkma로 명사 추출시 input_text 중복해서 들어가기 때문에 제거
if len(input_text_list) == len(noun_list) :
    text = input_text

else :
    [i for i in input_text_list if not i in noun_list or noun_list.remove(i)] #차집합인데 순서가 안 바뀜 
    text = ' '.join(noun_list)
print(text)

#Kkma 처음에 불러오는데 10초

스마트 폰


In [74]:
#슬로건 생성 모델 실행
import requests
import json


r = requests.post(
    'https://train-8dgtlge21881yafjrqb4-gpt2-train-teachable-ainize.endpoint.ainize.ai/predictions/gpt-2-ko-small-finetune', #슬로건 , 없는 정제된 5에포크
    headers = {'Content-Type' : 'application/json'
               },
    data=json.dumps({
  "text": text,
  "num_samples": 50,
  "length": 20
    }))

#슬로건 추가
slogan_list = []
for slogan in r.json():
    slogan = slogan.split('\n')[0]
    slogan = slogan.split(',')[1:]
    slogan = ', '.join(slogan)
    if slogan :
      slogan_list.append(slogan)
      print(slogan)
print(len(slogan_list),'개 완료')

#슬로건 데이터 200개 생성 25초

0 개 완료


In [58]:
#차집합 함수
def differ_sets(a,b) : 
    lst = list(set(a) - set(b))
    return lst
    

In [70]:
#영어 슬로건 따로 추출
import re

eng_list = []
for slogan in slogan_list :
    slogan_ = re.sub('[^A-Za-z가-힣]', '',slogan) #영어 한글만 남기기
    slogan_ = re.sub('[^가-힣]',' ', slogan_) #한글이 있으면 공백 채우기
    if slogan_.isspace():    #isalpha()는 영어 또는 한글 유무를 찾아서 안 됨
        eng_list.append(slogan)
        
print(eng_list)

#차집합 
kor_list = differ_sets(slogan_list, eng_list) #한국 슬로건만 있는 리스트

['Good break', 'Imagine possible', 'Go Go', 'The New Original Software', "What's Pick", 'Shift Delay on a spring', ' Tour seven', "It's Digital Success"]


In [14]:
#영어로는 유사도 분석이 힘들다.

# from sentence_transformers import SentenceTransformer, util
# import numpy as np
# import random

# #모델 불러오기
# model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# #유사도 비교할 리스트
# corpus = eng_list
# corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# #비교할 슬로건 선택 => 임시로 우선 정해서 정확도 확인
# queries = random.sample(eng_list, 3)

# top_k = 4
# for query in queries:
#     query_embedding = model.encode(query, convert_to_tensor=True)
#     cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
#     cos_scores = cos_scores.cpu()

#     #We use np.argpartition, to only partially sort the top_k results
#     top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k] 

#     print("\n======================\n")
#     print("Query:", query)
#     print("\nTop 5 most similar sentences in corpus:")

#     for idx in top_results[1:top_k]:
#         print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))




Query: I MAPPEED SUPER TOME

Top 5 most similar sentences in corpus:
THE BATTY BEAD STIGHT T (Score: 0.6062)
THE BEAUTY MAKE S (Score: 0.5947)
KBLEITAL ME TOUR MY TH (Score: 0.5820)


Query: WHAT YOUR MAKE

Top 5 most similar sentences in corpus:
WEEK YOUR BUTIONAR O (Score: 0.6320)
THE BEAUTY MAKE S (Score: 0.5894)
MY CALLE (Score: 0.5767)


Query: We lovers that wear. all. Mind for m

Top 5 most similar sentences in corpus:
I MAKE YOUR MILK,  CHARE (Score: 0.3331)
WE PRISME A COLLECTION (Score: 0.2203)
I MAPPEED SUPER TOME (Score: 0.2047)


In [38]:
#문장 유사도
#개별 추출
from sentence_transformers import SentenceTransformer, util
import numpy as np
import random

#모델 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

#유사도 비교할 리스트
corpus = kor_list
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)


#비교할 슬로건 선택 
no_sim_list = [] #관련 없는 슬로건 추출
n = 0
try : #n이 증가하지 않을 경우 무한루프? 
    while n < 5 :
        query = random.sample(kor_list, 1)
        print('='*40)
        print("Query : ", query)
        kor_list = differ_sets(kor_list, query)  #kor_list에서 query를 제거 
        
        #코사인 유사도 사용하여 5개 유사한 슬로건 찾기
        top_k = 6 #query 포함 top 5개
        query_embedding = model.encode(query, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k] # np 사용 이유 : 순위를 순서대로 맞추기 위함
        
        #민감도 비교하기 위한 유사도 더하기      
        sum = 0
        for idx in top_results[1:top_k]:
            sum += cos_scores[idx]
        f_sum = float(sum)/5 #tensor to float
        print(f_sum)
        
        #사용자 인풋 민감도 비교    
        sim_list = [] #유사 슬로건 담을 리스트
        if f_sum >= input_sim / 100 :
            for idx in top_results[0:top_k-1]:
                sim_list.append(corpus[idx].strip())
            print(sim_list)
            kor_list = differ_sets(kor_list, sim_list)
            n += 1
            #print(len(kor_list))
            
        else : 
            no_sim_list.append(query)
            print('관련이 없는 슬로건 데이터 추가')
  
                
                
except :
    print('데이터가 부족합니다.')
    
print('완료')
print(no_sim_list)

#슬로건 25개 보여주는데 40초 / 모델 불러들어오는 시간 포함 => 다시 시작되면 2~5초

Query :  ['이제 피부가 깨끗해졌습니다']
0.7614649772644043
['이제 피부가 깨끗해졌습니다', '이제부터 피부도 피부다', '이제 피부가 알아서 알아서', '이제 피부를 지켜주세요', '오늘의 피부']
Query :  ['그것은 아름다움을 만드는 시간입니다']
0.6298552989959717
['그것은 아름다움을 만드는 시간입니다', '당신의 모든 순간이 아름답습니다', '모든 순간을 아름답게', '당신은 지금 이 순간에도 뷰티스타일', '아름답게 예뻐지자']
Query :  ['수고했다,  나만의 스타일리스트']
0.45457162857055666
관련이 없는 슬로건 데이터 추가
Query :  ['나를 사랑하는 아름다움']
0.7339666843414306
['나를 사랑하는 아름다움', '나는 아름답다', '아름답게 예뻐지자', '당신만의 아름다움', '나는 당신의 아름다움에 감탄한다']
Query :  ['나만의 화장실을 꾸립니다']
0.39769206047058103
관련이 없는 슬로건 데이터 추가
Query :  ['화장품 브랜드,  립케어']
0.4849266052246094
관련이 없는 슬로건 데이터 추가
Query :  ['대한민국 1등']
0.6913610458374023
['대한민국 1등', '대한민국 NO.1 TSG피부', '대한민국 No.1 안티에이징', '대한민국 No.1 수분 밸런스', '대한민국 NO.1 뷰티브랜드']
Query :  ['화사한 피부를 위하여']
0.7406060218811035
['화사한 피부를 위하여', '미세먼지로부터 피부를 지켜주는', '피부엔 스킨케어가 필요할 때', '피부 보습이 필요한 순간', '이제부터 피부도 피부다']
완료
[['수고했다,  나만의 스타일리스트'], ['나만의 화장실을 꾸립니다'], ['화장품 브랜드,  립케어']]


In [ ]:
#문장 유사도
#개별 추출
from sentence_transformers import SentenceTransformer, util
import numpy as np
import random

#모델 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

#유사도 비교할 리스트
corpus = kor_list
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)


#비교할 슬로건 선택 
confirm_list = [] # 슬로건 추출
n = 0
try : #n이 증가하지 않을 경우 무한루프? 
    while n < 10 :
        query = random.sample(kor_list, 1)
        # print('='*40)
        # print("Query : ", query)
        kor_list = differ_sets(kor_list, query)  #kor_list에서 query를 제거 
        
        #코사인 유사도 사용하여 5개 유사한 슬로건 찾기
        top_k = 6 #query 포함 top 5개
        query_embedding = model.encode(query, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k] # np 사용 이유 : 순위를 순서대로 맞추기 위함
        
        #민감도 비교하기 위한 유사도 더하기      
        sum = 0
        for idx in top_results[1:top_k]:
            sum += cos_scores[idx]
        f_sum = float(sum)/5 #tensor to float
        # print(f_sum)
        
        #사용자 인풋 민감도 비교    
        if f_sum >= input_sim / 100 :
            confirm_list.append(query)
            n += 1
            #print(len(kor_list))
            
        else : 
            no_sim_list.append(query)
            print('관련이 없는 슬로건 데이터 추가')
  
                
                
except :
    print('데이터가 부족합니다.')
    
print('완료')
print(confirm_list)

#슬로건 25개 보여주는데 40초 / 모델 불러들어오는 시간 포함 => 다시 시작되면 2~5초